In [1]:
# syft absolute
import syft as sy
from syft import ActionObject
from syft.client.syncing import compare_states
from syft.client.syncing import resolve

In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=f4daad706cf74d93b461807a6c618175 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=4ebff4b1132347fcb6ef2e2137f55c05 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

/Users/eelco/dev/PySyft/packages/syft/src/syft/types/syft_object.py:588: TypeHintWarning: Skipping type check against 'ServiceRole'; this looks like a string-form forward reference imported from another module
  check_type(value, var_annotation)


Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 14.02it/s]

Uploading: numpy-data
syft_node_location=None syft_client_verify_key=None output={'id': <UID: d8667172a355431ebc9df5b51fd75832>, 'name': 'numpy-data', 'description': syft.service.dataset.dataset.MarkdownDescription, 'contributors': {syft.service.dataset.dataset.Contributor}, 'data_subjects': [], 'node_uid': <UID: 8212e6797fde4c3fba4fc53ab555a886>, 'action_id': <UID: 98736ea2cd8a44fea7e1bafa0d390ba9>, 'data': array([15, 16, 17, 18, 19]), 'mock': array([10, 11, 12, 13, 14]), 'shape': (5,), 'mock_is_real': True, 'created_at': syft.types.datetime.DateTime, 'uploader': syft.service.dataset.dataset.Contributor} node=Domain: test_h - 8212e6797fde4c3fba4fc53ab555a886 - domain

Services:
ActionService, BlobStorageService, CodeHistoryService, DataSubjectMemberService, DataSubjectService, DatasetService, EnclaveService, JobService, LogService, MetadataService, MigrateStateService, NetworkService, NotificationService, NotifierService, OutputService, PolicyService, ProjectService, QueueService, Req

SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 27.15it/s]

Uploading: numpy-data
syft_node_location=None syft_client_verify_key=None output={'id': <UID: f37a1e445f4546f2ba673d7373aef6db>, 'name': 'numpy-data', 'description': syft.service.dataset.dataset.MarkdownDescription, 'contributors': {syft.service.dataset.dataset.Contributor}, 'data_subjects': [], 'node_uid': <UID: 8a1c04544655402190588aec30079bc3>, 'action_id': <UID: 4c87ac79799d4b6eaff055b445d51e68>, 'data': Pointer:
None, 'mock': array([0, 1, 2, 3, 4]), 'shape': (5,), 'mock_is_real': True, 'created_at': syft.types.datetime.DateTime, 'uploader': syft.service.dataset.dataset.Contributor} node=Domain: test_l - 8a1c04544655402190588aec30079bc3 - domain

Services:
ActionService, BlobStorageService, CodeHistoryService, DataSubjectMemberService, DataSubjectService, DatasetService, EnclaveService, JobService, LogService, MetadataService, MigrateStateService, NetworkService, NotificationService, NotifierService, OutputService, PolicyService, ProjectService, QueueService, RequestService, Settin

In [8]:
dataset_storage_permissions = node_low.python_node.get_service(
    "datasetservice"
).stash.partition.storage_permissions
dataset_id = client_low.datasets[0].id

assert dataset_storage_permissions[dataset_id] == {node_low.python_node.id}

# Data Scientist: make requests

In [9]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    print("Computing mean...")
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/7567811fb3e244b8a70d689b0cec0607.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=2 image_uid=b4ef4707122f41b99e99fa8394486bbe in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_2089: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_2089
Computing mean...
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

SyftInfo: Node Landed!


In [10]:
client_low_ds.code.request_code_execution(compute_mean)

syft.service.request.request.Request

In [12]:
client_low_ds.code.compute_mean(data=data_low)

SyftError: Execution denied, Please wait for the code to be approved

## Data Owner: Sync to high side

In [13]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

In [14]:
assert (
    set(low_state.objects.keys())
    == set(low_state.permissions.keys())
    == set(low_state.storage_permissions.keys())
)

In [15]:
low_state

syft.service.sync.sync_state.SyncState

In [16]:
diff_state = compare_states(low_state, high_state)

In [17]:
diff_state

syft.service.sync.diff_state.NodeDiff

In [18]:
resolved_state_low, resolved_state_high = resolve(diff_state)

LOW SIDE STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #96cc905a76c841258c2a222231ea4f80:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]
worker_pool_name: default-pool

    ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #4257adda7b0a479ab7d53be332e1337d:
    approved: False
    status_dict: {
    ‎ ‎ ‎ ‎ node_id=<UID: 8a1c04544655402190588aec30079bc3>
        verify_key=1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367
        node_name='test_l': (<UserCodeStatus.PENDING: 'pending'>, '')
    }



HIGH SIDE STATE:

No high side changes.

Do you want to keep the low state or the high state for these objects? choose 'low' or 'high'


 low


Decision: Syncing 2 objects from low side


LOW SIDE STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

REQUEST #148f3b997dca4d21a5b539eeb949e8ad:
request_time: 2024-03-12 14:30:59
updated_at: None
status: RequestStatus.PENDING
changes: [
‎ ‎ ‎ ‎ syft.service.request.request.UserCodeStatusChange
]
requesting_user_verify_key: 869f868281dba1a96d0057208629a73076777dda142aefa26f49b050a80729a5

    ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #96cc905a76c841258c2a222231ea4f80:
    service_func_name: compute_mean
    input_owners: [
    ‎ ‎ ‎ ‎ test_l
    ]
    code_status: [
    ‎ ‎ ‎ ‎ Node: test_l, Status: pending
    ]
    worker_pool_name: default-pool



HIGH SIDE STATE:

No high side changes.

Do you want to keep the low state or the high state for these objects? choose 'low' or 'high'


 low


Decision: Syncing 2 objects from low side




In [19]:
print("Resolved state low side")
print(resolved_state_low)
print()
print("Resolved state high side")
print(resolved_state_high)

Resolved state low side
ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)

Resolved state high side
ResolvedSyncState(
  create_objs=[{NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}, syft.service.code.user_code.UserCode, syft.service.request.request.Request],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)


In [20]:
assert len(resolved_state_high.new_permissions) == 0

new_objs = resolved_state_high.create_objs
assert {o.id for o in new_objs} == {
    sp.uid for sp in resolved_state_high.new_storage_permissions
}

In [21]:
client_low.apply_state(resolved_state_low)

/Users/eelco/dev/PySyft/packages/syft/src/syft/types/syft_object.py:588: TypeHintWarning: Skipping type check against 'ServiceRole'; this looks like a string-form forward reference imported from another module
  check_type(value, var_annotation)


SyftSuccess: Synced 0 items

In [22]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 3 items

In [23]:
request_storage_permissions = node_high.python_node.get_service(
    "requestservice"
).stash.partition.storage_permissions
request_id = client_high.requests[0].id
node_high_id = node_high.python_node.id

assert node_high_id in request_storage_permissions[request_id]

# Data Owner: Run code high-side

In [25]:
data_high = client_high.datasets[0].assets[0]

In [26]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = 1638cafcef8c48cc8ea6427b2dd1759a
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

12/03/24 16:35:26 FUNCTION LOG (1638cafcef8c48cc8ea6427b2dd1759a): Computing mean...


In [27]:
# wait for the result
job_high.wait().get()

17.0

In [28]:
job_info = job_high.info(public_metadata=True, result=True)

request = client_high.requests[0]
result_obj = job_high.result

In [29]:
job_info

syft.service.job.job_stash.JobInfo

In [30]:
# syft absolute
from syft import SyftError
from syft import SyftSuccess

# Accepting the result directly gives an error
accept_res = request.accept_by_depositing_result(result_obj)
assert isinstance(accept_res, SyftError)

In [31]:
accept_res = request.accept_by_depositing_result(job_info)

assert isinstance(accept_res, SyftSuccess)
accept_res

Approving request for domain test_h
returning existing job
setting permission
None
None
Approving request for domain test_h
ADDING PERMISSION [READ: 143950cc55b942f28612f6d2f1e56362 as 869f868281dba1a96d0057208629a73076777dda142aefa26f49b050a80729a5] 143950cc55b942f28612f6d2f1e56362
Job(1638cafcef8c48cc8ea6427b2dd1759a) Setting new result 143950cc55b942f28612f6d2f1e56362 -> 143950cc55b942f28612f6d2f1e56362


SyftSuccess: Request submitted for updating result.

In [33]:
# Need to refresh Job because we overwrite result
job_high = client_high.jobs[0]

action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service(
    "blobstorageservice"
).stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_high.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]
)

In [34]:
assert action_store_high.storage_permissions[job_high.result.id.id] == {
    node_high.python_node.id
}

## Data Owner: Sync back to low-side

In [36]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

print(high_state)

syft.service.sync.sync_state.SyncState


In [38]:
diff_state_2 = compare_states(low_state, high_state)

In [39]:
resolved_state_low, resolved_state_high = resolve(diff_state_2)

LOW SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #96cc905a76c841258c2a222231ea4f80:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]

    ―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #4257adda7b0a479ab7d53be332e1337d:
    status_dict: {
    ‎ ‎ ‎ ‎ node_id=<UID: 8a1c04544655402190588aec30079bc3>
        verify_key=1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367
        node_name='test_l': (<UserCodeStatus.PENDING: 'pending'>, '')
    }



HIGH SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #96cc905a76c841258c2a222231ea4f80:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_h, Status: approved
]

    ―――― DIFF ――――――――

 high


Decision: Syncing 2 objects from high side


LOW SIDE STATE:

―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

REQUEST #148f3b997dca4d21a5b539eeb949e8ad:
node_uid: 8a1c04544655402190588aec30079bc3
changes: [
‎ ‎ ‎ ‎ syft.service.request.request.UserCodeStatusChange
]
history: [
]

    ―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #96cc905a76c841258c2a222231ea4f80:
    service_func_name: compute_mean
    input_owners: [
    ‎ ‎ ‎ ‎ test_l
    ]
    code_status: [
    ‎ ‎ ‎ ‎ Node: test_l, Status: pending
    ]



HIGH SIDE STATE:

―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

REQUEST #148f3b997dca4d21a5b539eeb949e8ad:
node_uid: 8212e6797fde4c3fba4fc53ab555a886
changes: [
‎ ‎ ‎ ‎ syft.service.request.request.UserCodeStatusChange
‎ ‎ ‎ ‎ syft.service.request.request.ActionStoreChange
]
history: [
‎ ‎ ‎ ‎ syft.service.

 high


Decision: Syncing 2 objects from high side


LOW SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #96cc905a76c841258c2a222231ea4f80:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]



HIGH SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #96cc905a76c841258c2a222231ea4f80:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_h, Status: approved
]

    ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    JOB #1638cafcef8c48cc8ea6427b2dd1759a:
    id: 1638cafcef8c48cc8ea6427b2dd1759a
    result: ActionDataEmpty <<class 'numpy.number'>>
    resolved: True
    progress:
    creation_time: 2024-03-12 16:35:25.168746

        ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――

...

This batch of updates contains new private objects on the high side that you may want     to share with user 869f868281dba1a96d0057208629a73076777dda142aefa26f49b050a80729a5.
You currently have the following private objects:

NumpyScalarObject #143950cc55b942f28612f6d2f1e56362
SyftLog #46b03e72ad7247a2b342cbe0f8bccc7a

Do you want to share some of these private objects? If so type the first 3 characters of the id e.g. 'abc'.
If you dont want to share any more private objects, type "no"



 143




Setting permissions for NumpyScalarObject #143950cc55b942f28612f6d2f1e56362 to share with 869f868281dba1a96d0057208629a73076777dda142aefa26f49b050a80729a5,
this will become effective when you call client.apply_state(<resolved_state>))

You currently have the following private objects:

SyftLog #46b03e72ad7247a2b342cbe0f8bccc7a

Do you want to share some of these private objects? If so type the first 3 characters of the id e.g. 'abc'.
If you dont want to share any more private objects, type "no"



 no


Decision: Syncing 5 objects from high side




In [40]:
print(resolved_state_low)
print()
print(resolved_state_high)

ResolvedSyncState(
  create_objs=[syft.service.output.output_service.ExecutionOutput, Pointer:
17.0, syft.service.job.job_stash.Job, syft.service.log.log.SyftLog],
  update_objs=[{NodeIdentity <name=test_h, id=8212e679, 🔑=d311a667>: (<UserCodeStatus.APPROVED: 'approved'>, '')}, syft.service.request.request.Request],
  delete_objs=[]
  new_permissions=[[READ: 143950cc55b942f28612f6d2f1e56362 as 869f868281dba1a96d0057208629a73076777dda142aefa26f49b050a80729a5], [READ: 1638cafcef8c48cc8ea6427b2dd1759a as 869f868281dba1a96d0057208629a73076777dda142aefa26f49b050a80729a5]]
)

ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)


In [41]:
res = client_low.apply_state(resolved_state_low)
res

SyftSuccess: Synced 6 items

In [42]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 0 items

In [43]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_low.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]
)

In [46]:
result_storage_permissions = node_low.python_node.get_service(
    "actionservice"
).store.storage_permissions[job_high.result.id.id]
assert len(result_storage_permissions) == 1

log_storage_permissions = node_low.python_node.get_service(
    "logservice"
).stash.partition.storage_permissions[job_high.log_id]
assert len(log_storage_permissions) == 1

In [52]:
client_low.code.compute_mean.jobs[0].log_id

<UID: 46b03e72ad7247a2b342cbe0f8bccc7a>

In [60]:
node_low.python_node.get_service("logservice").stash.partition.data[
    client_low.code.compute_mean.jobs[0].log_id
]

```python
class SyftLog:
  id: str = 46b03e72ad7247a2b342cbe0f8bccc7a
  stdout: str = ""
  stderr: str = ""

```

In [61]:
node_low.python_node.get_service("logservice").stash.partition.storage_permissions

{<UID: 46b03e72ad7247a2b342cbe0f8bccc7a>: set()}

# Run code low

## Run

In [59]:
client_low_ds._fetch_api(client_low.credentials)

In [60]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

```python
Pointer
```
17.0

In [61]:
res_low.id == job_high.result.id

True

In [62]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id.id
    == job_high.result.id.id
    == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [63]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

```python
Pointer
```
17.0

In [64]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id.id
    == job_high.result.id.id
    == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [65]:
private_high.mean()

17.0

In [66]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)
job_low

SyftWarning: There are existing jobs for this user code, returning the latest one

```python
class Job:
    id: UID = a3746c4c91344ff4acb13299806173c8
    status: completed
    has_parent: False
    result: ActionDataEmpty <<class 'numpy.number'>>
    logs:

0 Log 5344f766365a4f11b0edf837b732b447 not available
JOB COMPLETED
    
```

In [68]:
job_low.wait().get()

17.0

In [69]:
job_low.logs()

Log 5344f766365a4f11b0edf837b732b447 not available


In [70]:
job_high.result.id

<LineageID: b22fb7d828af42c5844e0a8af46571dc - 1570570224294398004>

In [71]:
job_high.result.as_empty().id

<LineageID: b22fb7d828af42c5844e0a8af46571dc - 1570570224294398004>

In [72]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert (
    job_low.result.syft_blob_storage_entry_id
    == job_high.result.syft_blob_storage_entry_id
)

In [73]:
job_high.result.id

<LineageID: b22fb7d828af42c5844e0a8af46571dc - 1570570224294398004>

In [74]:
job_low.result.id

<LineageID: b22fb7d828af42c5844e0a8af46571dc - 1570570224294398004>

In [75]:
job_high

```python
class Job:
    id: UID = a3746c4c91344ff4acb13299806173c8
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 Computing mean...
JOB COMPLETED
    
```